# 第六章 PyTorch进阶训练技巧

## 6.1 自定义损失函数
随着深度学习的发展, 传统的在`torch.nn`中收录的损失函数慢慢无法完全满足我们的需求. 对于一下非通用的模型, 有许多独特的损失函数需要我们自定义损失函数来提升模型效果.

因此学习自定义损失函数是非常重要的一部分

### 6.1.1 以函数定义
损失函数同样是一个函数, 因此可以以函数的形式直接定义

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def my_loss(output, target):
    loss = torch.mean((output-target)**2)
    return loss

### 6.1.2 以类定义
我们可以把损失函数看作神经网络的一层, 通过```nn.Module```来定义损失函数类.

我们如果看每一个损失函数的继承关系我们就可以发现`Loss`函数部分继承自`_loss`, 部分继承自`_WeightedLoss`, 而`_WeightedLoss`继承自`_loss`, ` _loss`继承自 **nn.Module**.

以DiceLoss的实现为例进行说明

DiceLoss的定义如下:
$$
DSC = \frac{2|X∩Y|}{|X|+|Y|}
$$

In [3]:
class DiceLoss(nn.Module):
    def __init__(self, weight = None, size_average = True):
        super(DiceLoss, self).__init__()
        
    def forward(self, inputs, targets, smooth=1):
        inputs = F.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targetsts.sum() + smooth)
        return 1-dice
    
criterion = DiceLoss()
# inputs = 0
# targets = 0
# loss = criterion(inputs, targets)
        

### 注意事项
在涉及到数学运算的过程时, 最好使用PyTorch的张量计算接口, 这样就可以实现自动求导功能并可以直接调用cuda

## 6.2 动态调整学习率
在深度学习中, 一个固定的学习速率会影响后期的训练效果. 在多轮训练后, 就会出现准确率震荡或者loss不再下降的情况.

因此, 我们可以设定一种合理的学习率衰减策略来改善现象, 以此提高精度. 这种方式被称为scheduler.

### 6.2.1 使用官方scheduler
官方的scheduler封装在`torch.optim.lr_scheduler`模块中

下为官方的实例代码

In [4]:
# 选择一种优化器
optimizer = torch.optim.Adam(...) 
# 选择上面提到的一种或多种动态调整学习率的方法
scheduler1 = torch.optim.lr_scheduler.... 
scheduler2 = torch.optim.lr_scheduler....
...
schedulern = torch.optim.lr_scheduler....
# 进行训练
for epoch in range(100):
    train(...)
    validate(...)
    optimizer.step()
    # 需要在优化器参数更新之后再动态调整学习率
	scheduler1.step() 
	...
    schedulern.step()

SyntaxError: invalid syntax (Temp/ipykernel_27164/3568393739.py, line 4)

### 6.2.2 自定义scheduler
我们同样可以自定义函数进行学习速率的调整

In [ ]:
# 定义一个每三十轮下降为原来的1/10
def adjust_learning_rate(optimizer,epoch):
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

然后通过这个函数, 我们可以调用函数实现学习率的变化, 具体方式同调用官方的方法.

## 6.3 模型微调
### 6.3.1 背景知识
#### 为什么要模型微调
对于很多应用场景, 一个经典的固定模型都会因为数据集和模型的不契合产生过拟合等现象. 但是要重新收集相匹配的数据又非常困难, 因此我们可以通过将源数据集学到的知识迁移到目标数据集上. 这种方法是**迁移学习**的一种应用.

因此, 我们就可以通过找一个同类的模型, 将模型通过训练调整一下参数, 然后就可以应用在自己的场景上.

PyTorch中有许多预训练好的网络模型(VGG, ResNet系列, mobilenet系列)

### 6.3.2 模型微调的流程
1. 拥有一个预训练完成的网络模型, 又称源模型
2. 创建一个目标模型, 它复制了源模型上除了输出层歪的所有模型设计和参数.
3. 为目标模型添加一个输出层, 这个输出层需要适应目标数据集, 然后随机初始化模型参数
4. 训练目标模型, 将输出层完全训练完后, 再微调其余层的参数即可

![finetune](./figures/finetune.png)

### 6.3.3 使用已有模型的注意事项
1. PyTorch模型扩展为`.pt`或`.pth`
2. 我们可以在[下载网址](https://github.com/pytorch/vision/tree/master/torchvision/models)中查看自己的模型里面`model_urls`, 然后手动下载.
3. 默认下载地址是在`Windows`下就是`C:\Users\<username>\.cache\torch\hub\checkpoint`. 我们可以通过使用 [`torch.utils.model_zoo.load_url()`](https://pytorch.org/docs/stable/model_zoo.html#torch.utils.model_zoo.load_url)设置权重的下载地址.

### 6.3.4 训练特定层
在我们只想训练新加入的初始化层的情况下, 我们就可以通过设置`requires_grad = False`来冻结部分层.

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

这样的话就可以达成只训练指定模型的特定层的目标.

## 6.4 半精度训练
半精度计算可以减少显存占用. 使得显卡可以同时加载更多数据进行计算.

#### 6.4.1 半精度训练的设置
- import autocast
- 模型设置
    - 使用装饰器的方法进行forward函数的装饰
- 训练过程放在`with autocast()`后面